Rand Agent Table with RNN for Time Series:

MODEL 4


* For agents with type Car only
* For all scenes 
* 20 most frequent agents only
* zero-padded (front padding included)

Look at this for more info on multiple parallel series with RNN: https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [ ]:
import keras
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
folderpath = "*"
folderpath = "C:\\Users\\Benson\\Desktop"
file = folderpath + "\\lyftlong\\rand_agents_table0.csv"
at = pd.read_csv(file)
file = folderpath + "\\lyftlong\\rand_frames_table1.csv"
ft = pd.read_csv(file)
file = folderpath +  "\\lyftlong\\rand_scenes_table1.csv"
st = pd.read_csv(file)

at = at.merge(ft[['frame_index', 'scene_index']], on='frame_index')

In [ ]:
def get_n_most_frequent(n: int, freq_track_ids):
    """ get's track ids of the n most frequent entities in a scene."""
    freq_track_ids = freq_track_ids.sort_values('frame_index', ascending=False)
    return freq_track_ids['track_id'].iloc[:20].values
def make_partitions(ar: np.array, step: int = 3) -> np.array:
    len_ar = len(ar)
    partitions = np.array([])
    for i in range(len_ar - step):
        X_partition = ar[i: i + step]
        y_partition = ar[i + step]
        partition = dict()
        partition['X'] = ((X_partition))
        partition['y'] = ((y_partition))
        partitions = np.append(partitions, partition)
    return partitions

In [ ]:
# FEATURE: filter to only types with Car; uncomment the above to include all agent types
at = at[at.PERCEPTION_LABEL_CAR > 0.5] 

In [ ]:
atcols = ['centroid_x', 'centroid_y', 'track_id', 'frame_index', 'scene_index']
atrnn = at[atcols]

scene_indices = pd.unique(at['scene_index'])

In [ ]:
atrnn1s = atrnn[atrnn.scene_index == scene_indices[10]]

In [ ]:
def scene_rnn_datatable(scene_index):
    # part 1: getting 20 frequent track_ids
    atrnn1s = atrnn[atrnn.scene_index == scene_indices[10]]
    track_ids = pd.unique(atrnn1s['track_id'])
    freq_track_ids = atrnn1s.groupby(['track_id']).count().reset_index()
    NUM_ENTITIES = len(track_ids)
    # FEATURE: We will only take the 20 most frequent track ids. Set N_MOST_FREQUENT to NUM_ENTITIES if you don't want that
    N_MOST_FREQUENT = 20 # NUM_ENTITIES
    most_freq_track_ids = get_n_most_frequent(N_MOST_FREQUENT, freq_track_ids)

    # part 2: create sequence mapping
    atrnn1s2 = atrnn1s[atrnn1s.track_id.isin(most_freq_track_ids)]
    atrnn1s2 = atrnn1s2[['centroid_x','centroid_y', 'track_id', 'frame_index']]
    apis = atrnn1s2.pivot(index='frame_index', columns='track_id')
    # FEATURE: replace all NaN's with zeros
    apis = apis.fillna(0)
    
    # part 3: partition (list of dictionaries, with x and y)
    NSTEPS = 3
    apis_arr = apis.to_numpy()
    partitions = make_partitions(apis_arr, NSTEPS)
    return partitions

In [ ]:
partitions = []
for scene_index in scene_indices:
    partition = scene_rnn_datatable(scene_index)
    partitions.extend(partition)

In [ ]:
TRAIN_PROPORTION = 0.8
TRAIN_PROPORTION_INDEXER = int(len(partitions) * 0.8)

np.random.shuffle(partitions)

X_train = []
y_train = []
X_test = []
y_test = []
for i in range(len(partitions)): # [:TRAIN_PROPORTION_INDEXER]:
    partition = partitions[i]
    if i <= TRAIN_PROPORTION_INDEXER:
        X_train.append(partition['X'])
        y_train.append(partition['y'])
    else:
        X_test.append(partition['X'])
        y_test.append(partition['y'])        
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
n_steps = 3
n_features = (40)

act = None
retseq = True

#### define model
model = Sequential()
model.add(LSTM(100, activation=act, return_sequences=retseq, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(50, activation=act))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

In [ ]:
history = model.fit(X_train, y_train, epochs=20, verbose=1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])

In [ ]:
model_eval = model.evaluate(X_test, y_test, batch_size=100)
print(model_eval)

## SECTION LIMIT